# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 7 2022 8:30AM,241241,10,0085840420,ISDIN Corporation,Released
1,Jul 7 2022 8:30AM,241241,10,0085840060,ISDIN Corporation,Released
2,Jul 7 2022 8:30AM,241241,10,0085840331,ISDIN Corporation,Released
3,Jul 7 2022 8:30AM,241241,10,0085840329,ISDIN Corporation,Released
4,Jul 7 2022 8:30AM,241241,10,0085840432,ISDIN Corporation,Released
5,Jul 7 2022 8:30AM,241241,10,0085840480,ISDIN Corporation,Released
6,Jul 7 2022 8:30AM,241241,10,0085840599,ISDIN Corporation,Released
7,Jul 7 2022 8:30AM,241241,10,0085840610,ISDIN Corporation,Released
8,Jul 7 2022 8:30AM,241241,10,0085840611,ISDIN Corporation,Released
9,Jul 7 2022 8:30AM,241241,10,0085840612,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
51,241226,Released,2
52,241227,Released,3
53,241228,Released,1
54,241240,Released,1
55,241241,Released,11


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
241226,NaN,2.0
241227,NaN,3.0
241228,NaN,1.0
241240,NaN,1.0
241241,NaN,11.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
241145,0.0,4.0
241147,0.0,3.0
241148,0.0,1.0
241149,0.0,2.0
241154,0.0,55.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
241145,0,4
241147,0,3
241148,0,1
241149,0,2
241154,0,55


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,241145,0,4
1,241147,0,3
2,241148,0,1
3,241149,0,2
4,241154,0,55


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,241145,,4
1,241147,,3
2,241148,,1
3,241149,,2
4,241154,,55


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 7 2022 8:30AM,241241,10,ISDIN Corporation
11,Jul 6 2022 4:03PM,241240,16,"SHL Pharma, LLC"
12,Jul 6 2022 3:04PM,241228,10,Eywa Pharma Inc.
13,Jul 6 2022 2:46PM,241227,10,Emerginnova
16,Jul 6 2022 2:40PM,241225,10,"Methapharm, Inc."
36,Jul 6 2022 2:40PM,241226,10,"Methapharm, Inc."
38,Jul 6 2022 2:25PM,241224,19,"AdvaGen Pharma, Ltd"
97,Jul 6 2022 2:17PM,241223,10,ISDIN Corporation
106,Jul 6 2022 2:17PM,241219,10,ISDIN Corporation
134,Jul 6 2022 2:14PM,241222,19,"AdvaGen Pharma, Ltd"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jul 7 2022 8:30AM,241241,10,ISDIN Corporation,,11
1,Jul 6 2022 4:03PM,241240,16,"SHL Pharma, LLC",,1
2,Jul 6 2022 3:04PM,241228,10,Eywa Pharma Inc.,,1
3,Jul 6 2022 2:46PM,241227,10,Emerginnova,,3
4,Jul 6 2022 2:40PM,241225,10,"Methapharm, Inc.",,20
5,Jul 6 2022 2:40PM,241226,10,"Methapharm, Inc.",,2
6,Jul 6 2022 2:25PM,241224,19,"AdvaGen Pharma, Ltd",,59
7,Jul 6 2022 2:17PM,241223,10,ISDIN Corporation,,9
8,Jul 6 2022 2:17PM,241219,10,ISDIN Corporation,,28
9,Jul 6 2022 2:14PM,241222,19,"AdvaGen Pharma, Ltd",5,6


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 7 2022 8:30AM,241241,10,ISDIN Corporation,11,
1,Jul 6 2022 4:03PM,241240,16,"SHL Pharma, LLC",1,
2,Jul 6 2022 3:04PM,241228,10,Eywa Pharma Inc.,1,
3,Jul 6 2022 2:46PM,241227,10,Emerginnova,3,
4,Jul 6 2022 2:40PM,241225,10,"Methapharm, Inc.",20,
5,Jul 6 2022 2:40PM,241226,10,"Methapharm, Inc.",2,
6,Jul 6 2022 2:25PM,241224,19,"AdvaGen Pharma, Ltd",59,
7,Jul 6 2022 2:17PM,241223,10,ISDIN Corporation,9,
8,Jul 6 2022 2:17PM,241219,10,ISDIN Corporation,28,
9,Jul 6 2022 2:14PM,241222,19,"AdvaGen Pharma, Ltd",6,5


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 7 2022 8:30AM,241241,10,ISDIN Corporation,11,
1,Jul 6 2022 4:03PM,241240,16,"SHL Pharma, LLC",1,
2,Jul 6 2022 3:04PM,241228,10,Eywa Pharma Inc.,1,
3,Jul 6 2022 2:46PM,241227,10,Emerginnova,3,
4,Jul 6 2022 2:40PM,241225,10,"Methapharm, Inc.",20,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jul 7 2022 8:30AM,241241,10,ISDIN Corporation,11.0,NaN
1,Jul 6 2022 4:03PM,241240,16,"SHL Pharma, LLC",1.0,NaN
2,Jul 6 2022 3:04PM,241228,10,Eywa Pharma Inc.,1.0,NaN
3,Jul 6 2022 2:46PM,241227,10,Emerginnova,3.0,NaN
4,Jul 6 2022 2:40PM,241225,10,"Methapharm, Inc.",20.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 7 2022 8:30AM,241241,10,ISDIN Corporation,11.0,0.0
1,Jul 6 2022 4:03PM,241240,16,"SHL Pharma, LLC",1.0,0.0
2,Jul 6 2022 3:04PM,241228,10,Eywa Pharma Inc.,1.0,0.0
3,Jul 6 2022 2:46PM,241227,10,Emerginnova,3.0,0.0
4,Jul 6 2022 2:40PM,241225,10,"Methapharm, Inc.",20.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,8441598,546.0,3.0
12,723561,5.0,9.0
16,964757,4.0,3.0
19,723661,66.0,5.0
20,723587,59.0,2.0
21,241183,0.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,8441598,546.0,3.0
1,12,723561,5.0,9.0
2,16,964757,4.0,3.0
3,19,723661,66.0,5.0
4,20,723587,59.0,2.0
5,21,241183,0.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,546.0,3.0
1,12,5.0,9.0
2,16,4.0,3.0
3,19,66.0,5.0
4,20,59.0,2.0
5,21,0.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,546.0
1,12,Released,5.0
2,16,Released,4.0
3,19,Released,66.0
4,20,Released,59.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20,21
Status,,,,,,
Executing,3.0,9.0,3.0,5.0,2.0,1.0
Released,546.0,5.0,4.0,66.0,59.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20,21
0,Executing,3.0,9.0,3.0,5.0,2.0,1.0
1,Released,546.0,5.0,4.0,66.0,59.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20,21
0,Executing,3.0,9.0,3.0,5.0,2.0,1.0
1,Released,546.0,5.0,4.0,66.0,59.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()